In [ ]:
system("conda install -y r-sf=0.9_8")
system("conda install -y r-terra=1.2_10")

In [ ]:
# Title: Fig1 ----
# Arthur BARREAU: 06 Février 2025

# 1. LOAD LIBRARIES ----
library(terra)      
library(sf)        

# 2. LOAD DATA ----
# 2.1 DOWNLOAD AND LOAD ASD ----
output_dir <- "asd"
dir.create(output_dir, showWarnings = FALSE)

urls_asd <- c(
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/asd-shapefile-EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/asd-shapefile-EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/asd-shapefile-EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/asd/asd-shapefile-EPSG4326.prj"
)

destfiles_asd <- file.path(output_dir, basename(urls_asd))

for (i in seq_along(urls_asd)) {
  download.file(urls_asd[i], destfiles_asd[i], mode = "wb")
}

ASDs <- st_read(file.path(output_dir, "asd-shapefile-EPSG4326.shp"))

# Reproject ASD to the correct CRS (EASE-Grid 2.0 South)
ASDs <- st_transform(ASDs, crs = 6932)

# 2.2 DOWNLOAD AND LOAD CEMP ----
output_dir_cemp <- "cemp"
dir.create(output_dir_cemp, showWarnings = FALSE)

urls_cemp <- c(
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/main/geographical_data/cemp/cemp-shapefile-EPSG4326.prj"
)

destfiles_cemp <- file.path(output_dir_cemp, basename(urls_cemp))

for (i in seq_along(urls_cemp)) {
  download.file(urls_cemp[i], destfiles_cemp[i], mode = "wb")
}

CEMP <- st_read(file.path(output_dir_cemp, "cemp-shapefile-EPSG4326.shp"))

# Reproject CEMP to match ASDs' CRS
CEMP <- st_transform(CEMP, st_crs(ASDs))

# 2.3 DOWNLOAD AND LOAD BATHYMETRY ----
bathy_file <- "GEBCO2024_5000.tif"
options(timeout=180)
download.file("https://gis.ccamlr.org/geoserver/www/GEBCO2024_5000.tif", bathy_file, mode = "wb")
Bathy <- rast(bathy_file)

# 3. CREATE MAP OUTPUT ----
png(filename = "outputs/test_Fig1.png", width = 2200, height = 1800, res = 300)

# 4. MAP PLOTTING ----
Depth_cuts <- c(-8200 ,-7000 ,-6000 ,-5000, -4000, -3000, -1800, -1400, -1000,  -600,  -400 , -200  ,   0 ,   50  , 250   ,500)
Depth_cols <- c("#0000FF" ,"#061CFF", "#0C39FF", "#1256FF", "#1873FF" ,"#1E90FF", "#5CACEE", "#71BDF3", "#87CEFA", "#B0E2FF" ,"#D0EDFF", "#F0F8FF", "#FFFFFF", "#F2F2F2", "#E5E5E5")

# Plot bathymetry
plot(Bathy,breaks = Depth_cuts, col = Depth_cols, legend = FALSE, axes = FALSE, box = FALSE)

# Add ASD boundaries in red
plot(st_geometry(ASDs), add = TRUE, lwd = 0.75, border = 'red')

# Add CEMP sites as orange points
plot(st_geometry(CEMP), add = TRUE, pch = 21, bg = 'orange', col = 'black', cex = 1.5)

# 5. SAVE OUTPUT ----
dev.off()



